In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format='retina'

import time
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
import os
import itertools
import datetime
import random
import math

import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(color_codes=True)

plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False
sns.set_style("darkgrid", {"axes.facecolor": ".9"})
pd.set_option('max_columns', 1000) 

In [2]:
SAVE_PROCESSED_DATA = True

In [3]:
scriptDir = os.getcwd()
relPath = r"../Customer Segmentation/"
sampleFilePath = os.path.join(scriptDir, relPath, 'CJ filtered.csv')
relPathOutput = r"../Customer Segmentation/output/"
outputFolderPath = os.path.join(scriptDir, relPathOutput)

In [4]:
# Load AirPartner Data
df = pd.read_csv(sampleFilePath, sep=',', header=0, quotechar='"', encoding='latin1')
print('Dataframe dimensions:', df.shape)

Dataframe dimensions: (8809, 7)


In [5]:
tab_info = pd.DataFrame(df.dtypes).T.rename(index={0:'column type'})
tab_info = tab_info.append(pd.DataFrame(df.isnull().sum()).T.rename(index={0:'null values (nb)'}))
tab_info = tab_info.append(pd.DataFrame(df.isnull().sum()/df.shape[0]*100).T.
                         rename(index={0:'null values (%)'}))
display(tab_info)

,Accountid,enqid,product,brokercountry,owner,qtq_firstflightdate,Grossprofit
column type,int64,int64,object,object,object,object,float64
null values (nb),0,0,0,0,0,0,0
null values (%),0,0,0,0,0,0,0


In [13]:
df = df[df['product'] != 'Tour Ops']
df = df[df['Grossprofit'] > 0]

# Exploratory Analysis
- Intermediate dataset groupbyed by flightid
- Final dataset grouped by customers
- K-means clustering
- Interpreting the clusters

## LRFMC principle
- LRFMC stands for 'LOAD-TIME-FFP_DATE', 'DAYS_FROM_LAST_TO_END', 'FLIGHT_COUNT_NUMBER', 'SEG_KM_SUM', and 'AVG_COMMISSION'.
- It is a customer segmentation technique that uses past purchase behaviour to divide customers into groups.

In [15]:
from scipy.stats import skew, norm, probplot, boxcox
from scipy.stats import kstest
import scipy.stats as stats

In [17]:
df.columns

Index(['Accountid', 'enqid', 'product', 'brokercountry', 'owner',
       'qtq_firstflightdate', 'Grossprofit'],
      dtype='object')

In [18]:
df['qtq_firstflightdate'] = pd.to_datetime(df['qtq_firstflightdate'])

print('qtq_firstflightdate:', df['qtq_firstflightdate'].min(), '->', df['qtq_firstflightdate'].max())

qtq_firstflightdate: 2016-02-01 00:00:00 -> 2020-05-15 23:00:00


In [ ]:
# Create a column that tells the Recency of deal
def reduction_data(data):
    data = data[['qtq_firstflightdate', 'Total Distance (NMi)', 'gross', 'companyid', 'enqid']]
    data['LOAD_TIME'] = pd.to_datetime('22-07-2019')
    data['DATEDIFF_BD'] = (data['LOAD_TIME'] - pd.to_datetime(data['booked date'])).dt.days

    L_agg = data.groupby('companyid')['DATEDIFF_BD'].agg({'L':'max'})
    R_agg = data.groupby('companyid')['DATEDIFF_BD'].agg({'R':'min'})
    F_agg = data.groupby('companyid')['booked date'].agg({'F': lambda x: x.size})
    M_agg = data.groupby('companyid')['Total Distance (NMi)'].agg({'M': 'sum'})
    C_agg = data.groupby('companyid')['gross'].agg({'C':'mean'})
    dataTransformed = (L_agg).join(R_agg).join(F_agg).join(M_agg).join(C_agg)
    return dataTransformed

dataTransformed = reduction_data(df)
# Transformed the timedelta64 to int 32

### Normality
- Histogram - Kurtosis and skewness
- Normal probablity plot - Data distribution should closely follow the diagonal that represents the normal distributionm

#### Length of time the customer relationship

In [ ]:
def qqplot(data, measure):
    fig = plt.figure(figsize=(20, 7))
    (mu, sigma) = norm.fit(data)
    
    fig1 = fig.add_subplot(121)
    sns.distplot(data, fit=norm, color='black')
    fig1.set_title(measure + 'Distribution (mu={:.2f} and sigma = {:.2f})'.format(mu, sigma), loc='center')
    fig1.set_xlabel(measure)
    fig1.set_ylabel('Frequency')
    
    fig2 = fig.add_subplot(122)
    res = probplot(data, plot=fig2)
    fig2.set_title(measure + 'Probability Plot (skewness): {:.6f} and kurtosis: {:.6f})'.format(data.skew(), data.kurt()), loc='center')
    
    plt.tight_layout()
    fig.savefig('L.png')
    plt.show()

qqplot(dataTransformed.L, 'Customer lifetime value')

- From the first graph above we can see that sales recency distribution is skewed, has a peak on the left and a long tail to the right. It deviates from normal distribution and is positively biased.

- From the Probability Plot, we could see that sales recency also does not align with the diagonal red line which represent normal distribution. The form of its distribution confirm that is a skewed right.

- With skewness negative of 0.055, we confirm the lack of symmetry and indicate that sales recency are skewed right, as we can see at the Sales Distribution plot, skewed left means that the left tail is long relative to the right tail. The skewness for a normal distribution is zero, and any symmetric data should have a skewness near zero. A distribution, or data set, is symmetric if it looks the same to the left and right of the center point.

- Kurtosis is a measure of whether the data are heavy-tailed or light-tailed relative to a normal distribution. That is, data sets with high kurtosis tend to have heavy tails, or outliers, and positive kurtosis indicates a heavy-tailed distribution and negative kurtosis indicates a light tailed distribution. So, with 1.11 of positive kurtosis sales recency are heavy-tailed and has some outliers.

#### Recency

In [ ]:
qqplot(dataTransformed.R, 'Recency')

- From the first graph above we can see that sales frequency distribution is skewed, has a peak on the left and a long tail to the right. It deviates from normal distribution and is positively biased.

- From the Probability Plot, we could see that sales frequency also does **not align with the diagonal and confirm that is a skewed right.

- With skewness positive of 12.1, we confirm the high lack of symmetry and with 249 Kurtosis indicates that is a heavy-tailed distribution and has outliers.

#### Frequency

In [ ]:
qqplot(dataTransformed.F, 'Frequency')

- From the first graph above we can see that sales amount distribution is skewed, has a peak on the left and a long tail to the right. It deviates from normal distribution and is positively biased.

- From the Probability Plot, we could see that sales amount also does not align with the diagonal, special on the right.

- With skewness positive of 19.3, we confirm the high lack of symmetry and with 478 Kurtosis indicates that is a too heavy-tailed distribution and has outliers, surely more than 10 very extreme.

#### Gross mileage

In [ ]:
qqplot(dataTransformed.M, 'Miles')

#### Average gross from each customer

In [ ]:
qqplot(dataTransformed.C, 'Average gross from each customer')

In [ ]:
len(dataTransformed[dataTransformed.R < 720])

In [ ]:
def plot_fit_pdf(data, data_inst, fig, units, title):
    from powerlaw import plot_ccdf, Fit, pdf, plot_pdf
    ax = fig.add_subplot(n_graphs,n_data,data_inst)
    
    fit = Fit(data, discrete=True, xmin=1, fit_Method="KS")
    fit.plot_pdf(ax=ax, color='r', label = 'Exponential Data')
    fit.power_law.plot_pdf(ax=ax, linestyle=':', color='g', label='Power-law fit')
    fit.exponential.plot_pdf(ax=ax, linestyle='--', color='b', label='Exponential fit')
    fit.lognormal.plot_pdf(ax=ax, linestyle='--', color='m', label='Lognormal fit')

    plt.xlabel(units)
    plt.title(title, fontsize = 12, fontweight = 'bold')
    plt.legend(loc='lower left', fancybox=True)

n_data = 1
n_graphs = 1
f = plt.figure(figsize=(10, 10))

unit = 'Customer lifetime value'
title = 'PDF of Lifetime Distribution'
plot_fit_pdf(dataTransformed.L, 1, f, unit, title)
plt.ylabel("$p(X\geq x)$")

plt.show()

In [ ]:
n_data = 1
n_graphs = 1
f = plt.figure(figsize=(10, 10))

unit2 = 'Recency'
title2 = 'PDF of Recency Distribution'
plot_fit_pdf(dataTransformed.R, 1, f, unit2, title2)

In [ ]:
n_data = 3
n_graphs = 1
f = plt.figure(figsize=(16, 6))

unit = 'Frequency'
title = 'PDF of Frequency Distribution'
plot_fit_pdf(dataTransformed.F, 1, f, unit, title)
plt.ylabel("$p(X\geq x)$")

unit = 'Monetary'
title = 'PDF of Monetary Distribution'
plot_fit_pdf(dataTransformed.M, 2, f, unit, title)
plt.ylabel("$p(X\geq x)$")

unit = 'Gross'
title = 'PDF of Gross Distribution'
plot_fit_pdf(dataTransformed.C, 3, f, unit, title)
plt.ylabel("$p(X\geq x)$")
f.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=.3, hspace=.2)
f.savefig('PDF.png')
plt.show()

In [ ]:
dataTransformed_summary = dataTransformed.describe(percentiles = [], include = 'all')

In [ ]:
print(kstest(dataTransformed.iloc[:,0], 'norm'))
print(kstest(dataTransformed.iloc[:,1], 'norm'))
print(kstest(dataTransformed.iloc[:,2], 'norm'))
print(kstest(dataTransformed.iloc[:,3], 'norm'))
print(kstest(dataTransformed.iloc[:,4], 'norm'))

### Log-transform to each vector
- A problem is the huge range of values each variable can take, particularly noticeable for the monetary amount variable.
- The log-transforamtion, along with the standardization, will ensure that the input to our algorithm is a homogenous set of scaled and transformed values.

In [ ]:
dataTransformed['L_log'] = np.log10(dataTransformed['L'])
dataTransformed['R_log'] = np.log10(dataTransformed['R'])
dataTransformed['F_log'] = np.log10(dataTransformed['F'])
dataTransformed['M_log'] = np.log10(dataTransformed['M'])
dataTransformed['C_log'] = np.log10(dataTransformed['C'])

feature_vector = ['L_log', 'R_log', 'F_log', 'M_log', 'C_log']
subset = dataTransformed[feature_vector]

## Feature Scaling
- One of the requirements for K-means is the mean centering of the variable values.
- Mean centering of a variable value means that we will replace the actual value of the variable with a standardized value, so that the variable has a mean of 0 and variance of 1. 

In [ ]:
subset = subset.fillna(method='pad')

In [ ]:
index = 0
for i in subset.iloc[:, 1]:
    if not np.isfinite(i):
        print(index, i)
    index += 1

subset.iloc[324, 1] = subset.iloc[:, 1].median()

In [ ]:
sns.boxenplot(data=subset[["L_log","R_log","F_log", "M_log", "C_log"]],
                    orient="h", palette="mako")
plt.xlabel("Log10 Normalized Raw Value")
plt.title("LRFMC Outlier Identification")

sns.despine()

### Discretization of RFM Features
- Quintile Normalization:  a method used to segment a features’ distributions into ﬁve ordinal groups

In [ ]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# subset['Scaled_L'] = scaler.fit_transform(subset[['L_log']])
# subset['Scaled_R'] = scaler.fit_transform(subset[['R_log']])
# subset['Scaled_F'] = scaler.fit_transform(subset[['F_log']])
# subset['Scaled_M'] = scaler.fit_transform(subset[['M_log']])
# subset['Scaled_C'] = scaler.fit_transform(subset[['C_log']])

In [ ]:
Quartile = subset.quantile(q=[0.25, 0.5, 0.75])

In [ ]:
dataTransformed.iloc[:, :5].quantile(q=[0.25, 0.5, 0.75])

In [ ]:
def R_class(x, p, data):
    if x <= data[p][0.25]:
        return 4
    elif x <= data[p][0.50]:
        return 3
    elif x <= data[p][0.75]: 
        return 2
    else:
        return 1

def other_class(x, p, data):
    if x <= data[p][0.25]:
        return 1
    elif x <= data[p][0.50]:
        return 2
    elif x <= data[p][0.75]: 
        return 3
    else:
        return 4   

In [ ]:
lrfmc_seg = subset

lrfmc_seg['L'] = lrfmc_seg['L_log'].apply(other_class, args=('L_log', Quartile))
lrfmc_seg['R'] = lrfmc_seg['R_log'].apply(R_class, args=('R_log', Quartile))
lrfmc_seg['F'] = lrfmc_seg['F_log'].apply(other_class, args=('F_log', Quartile))
lrfmc_seg['M'] = lrfmc_seg['M_log'].apply(other_class, args=('M_log', Quartile))
lrfmc_seg['C'] = lrfmc_seg['C_log'].apply(other_class, args=('C_log', Quartile))

# combine the scores to create a single score.
lrfmc_seg['LRFMC Class'] = lrfmc_seg.L.map(str) + lrfmc_seg.R.map(str) \
                        + lrfmc_seg.F.map(str) + lrfmc_seg.M.map(str) + lrfmc_seg.C.map(str)

In [ ]:
lrfmc_seg.sort_values(by=['LRFMC Class', 'C_log'], ascending=[True, False]).head()

In [ ]:
len(lrfmc_seg[(lrfmc_seg['C'] == 4) & (lrfmc_seg['R'] == 4)])

In [ ]:
lrfmc_seg['Total Score'] = lrfmc_seg['L'] + lrfmc_seg['R'] \
                        + lrfmc_seg['F'] + lrfmc_seg['M'] + lrfmc_seg['C']

lrfmc_table = lrfmc_seg[['L', 'R', 'F', 'M', 'C']]

In [ ]:
LRFMC_Model = dataTransformed.iloc[:, :5]
LRFMC_Model.columns = ['Lifetime', 'Recency', 'Frequency', 'SUM_Miles', 'Average GP']

lrfmc_seg = pd.merge(lrfmc_seg, LRFMC_Model, on='companyid')

In [ ]:
LRFMC_Model.quantile(q=[0.25, 0.5, 0.75])

In [ ]:
# lrfmc_seg.groupby('R')['Lifetime', 'Recency', 'Frequency', 'SUM_Miles', 'Average GP'].agg([np.mean, np.median, np.std])

## Model Construction
- K-means
- silhouette score
- Rader Map

In [ ]:
from scipy.cluster.hierarchy import linkage,dendrogram
from sklearn.cluster import KMeans 
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_samples, silhouette_score, calinski_harabaz_score
from gap_statistic import OptimalK
import sys

### The Elbow Method (WCSS)
- Elbow method to check the best fit K between 2-10

In [ ]:
X = lrfmc_table
distorsions = []
for k in range(1, 9):
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(X)
    distorsions.append(kmeans.inertia_)

fig = plt.figure(figsize=(7,6), dpi=72)
plt.plot(range(1, 9), distorsions, '-o', linewidth=0.9,  color="k", markerfacecolor='w')
plt.xlabel('Number of clusters')
plt.ylabel('Total Within Sum of Square')
plt.title('Normal/Quartile WCSS', fontweight='bold')
plt.tick_params(direction='out',width=2,length=4)
plt.axvline(x=3, color='black', linestyle = '--', alpha=0.7, linewidth=1)
plt.axvline(x=5, color='black', linestyle = '--', alpha=0.7, linewidth=1)
# fig.savefig('ElbowMethod.png')
plt.show()

### Silhouette analysis on K-Means clustering

In [ ]:
scores = []
for k in range(2, 11):
    labels = KMeans(n_clusters=k).fit(lrfmc_table).labels_
    score = silhouette_score(X, labels)
    scores.append(score)

In [ ]:
fig = plt.figure(figsize=(7,6), dpi=72)
plt.plot(range(2, 11), scores, '-o', linewidth=0.9)
plt.grid(True)
plt.xlabel('Number of clusters')
plt.ylabel('Average silhouette width')
plt.title('The Silhouette Method showing the optimal k', fontsize='large')
plt.axvline(x=5, color='black', linestyle = '--', alpha=0.7, linewidth=1)
plt.axvline(x=8, color='black', linestyle = '--', alpha=0.7, linewidth=1)
# fig.savefig('SilhouetteMethod.png')
plt.show()

### Gap Statisitic
- Construct the OptimalK class using the joblib backend

In [ ]:
optimalK = OptimalK(parallel_backend='rust')
optimalK

In [ ]:
def optimalK(data, nrefs=3, maxClusters=15):
    gaps = np.zeros((len(range(1, maxClusters)),))
    resultsdf = pd.DataFrame({'clusterCount':[], 'gap':[]})
    for gap_index, k in enumerate(range(1, maxClusters)):

        refDisps = np.zeros(nrefs)
        for i in range(nrefs):
                        
            km = KMeans(k)
            km.fit(lrfmc_table)
            
            refDisp = km.inertia_
            refDisps[i] = refDisp

        km = KMeans(k)
        km.fit(data)
        
        origDisp = km.inertia_
        gap = np.log(np.mean(refDisps)) - np.log(origDisp)
        gaps[gap_index] = gap
        
        resultsdf = resultsdf.append({'clusterCount':k, 'gap':gap}, ignore_index=True)

    return (gaps.argmax() + 1, resultsdf)  # Plus 1 because index of 0 means 1 cluster is optimal, index 2 = 3 clusters are optimal

In [ ]:
k, gapdf = optimalK(lrfmc_table, nrefs=5, maxClusters=11)
print ('Optimal k is: ', k)

In [ ]:
fig = plt.figure(figsize=(7,6), dpi=72)
plt.plot(gapdf.clusterCount, gapdf.gap, '-o', linewidth=0.9)
plt.xlabel('Number of clusters')
plt.ylabel('Gap statistic (k)')
plt.title('The Gap Method showing the optimal k', fontsize='large')
plt.axvline(x=5, color='black', linestyle = '--', alpha=0.7, linewidth=1)
plt.axvline(x=8, color='black', linestyle = '--', alpha=0.7, linewidth=1)
fig.savefig('output/GapMethod.png', dpi=500)
plt.show()

### Pareto Distribution of Cluster Validation

In [ ]:
Y = dataTransformed.iloc[:, :5]
distorsions = []
for k in range(1, 9):
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(Y)
    distorsions.append(kmeans.inertia_)

fig = plt.figure(figsize=(7,6), dpi=72)
plt.plot(range(1, 9), distorsions, '-o', linewidth=0.9,  color="k", markerfacecolor='w')
plt.xlabel('Number of clusters')
plt.ylabel('Total Within Sum of Square')
plt.title('Pareto/Raw WCSS', fontweight='bold')
# fig.savefig('ElbowMethod.png')
plt.show()

In [ ]:
Y1 = subset.iloc[:,:5]
distorsions = []
for k in range(1, 9):
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(Y1)
    distorsions.append(kmeans.inertia_)

fig = plt.figure(figsize=(7,6), dpi=72)
plt.plot(range(1, 9), distorsions, '-o', linewidth=0.9,  color="k", markerfacecolor='w')
plt.xlabel('Number of clusters')
plt.ylabel('Total Within Sum of Square')
plt.title('Normal/Raw WCSS', fontweight='bold')
plt.xticks(np.linspace(1, 9, 9))
plt.rcParams['xtick.direction'] = 'in'
# fig.savefig('ElbowMethod.png')
plt.show()

In [ ]:
Pareto_table = dataTransformed.iloc[:, :5]

In [ ]:
Pareto_Quartile = Pareto_table.quantile([0.25, 0.5, 0.75])

In [ ]:
Pareto_table['Lifetime'] = Pareto_table['L'].apply(other_class, args=('L', Pareto_Quartile))
Pareto_table['Recency'] = Pareto_table['R'].apply(R_class, args=('R', Pareto_Quartile))
Pareto_table['Frequency'] = Pareto_table['F'].apply(other_class, args=('F', Pareto_Quartile))
Pareto_table['Miles'] = Pareto_table['M'].apply(other_class, args=('M', Pareto_Quartile))
Pareto_table['GrossProfit'] = Pareto_table['C'].apply(other_class, args=('C', Pareto_Quartile))

In [ ]:
Y2 = Pareto_table.iloc[:,-5:]
distorsions = []
for k in range(1, 9):
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(Y2)
    distorsions.append(kmeans.inertia_)

fig = plt.figure(figsize=(7,6), dpi=72)
plt.plot(range(1, 9), distorsions, '-o', linewidth=0.9,  color="k", markerfacecolor='w')
plt.xlabel('Number of clusters')
plt.ylabel('Total Within Sum of Square')
plt.title('Pareto/Quartile WCSS', fontweight='bold')
# fig.savefig('ElbowMethod.png')
plt.show()

### Cluster center 
- look at the cluster center values after returning them to normal values from the log and scaled version

In [ ]:
from sklearn import metrics

#### number of clusters = 5

In [ ]:
kmodel = KMeans(n_clusters=5, init='k-means++', 
                n_init=10, max_iter=500, tol=1e-04,
                random_state=101)
kmodel_label = kmodel.fit_predict(lrfmc_table)

# Make clustering analysing by K-means alg
r1 = pd.Series(kmodel.labels_).value_counts() 
r2 = pd.DataFrame(kmodel.cluster_centers_) 

In [ ]:
r2.columns = ['L', 'R', 'F', 'M', 'C']

In [ ]:
y_pos = np.arange(len(r2.columns))

fig, axes = plt.subplots(1, 5, sharey=True)
a1 = sns.barplot(r2.columns, list(r2.iloc[0]), ax=axes[0], palette=('husl'))
a1.set_title(1)
for x, y in zip(y_pos, list(r2.iloc[0])):
    a1.text(x+0.001, y-0.03, int(y+0.5), ha='center', va='bottom')

a2 = sns.barplot(r2.columns, list(r2.iloc[1]), ax=axes[1], palette='husl')
a2.set_title(2)
for x, y in zip(y_pos, list(r2.iloc[1])):
    a2.text(x+0.001, y-0.03, int(y+0.5), ha='center', va='bottom')

a3 = sns.barplot(r2.columns, list(r2.iloc[2]), ax=axes[2], palette='husl')
a3.set_title(3)
for x, y in zip(y_pos, list(r2.iloc[2])):
    a3.text(x+0.001, y-0.03, int(y+0.5), ha='center', va='bottom')

a4 = sns.barplot(r2.columns, list(r2.iloc[3]), ax=axes[3], palette='husl')
a4.set_title(4)
for x, y in zip(y_pos, list(r2.iloc[3])):
    a4.text(x+0.001, y-0.03, int(y+0.5), ha='center', va='bottom')

a5 = sns.barplot(r2.columns, list(r2.iloc[4]), ax=axes[4], palette='husl')
a5.set_title(5)
for x, y in zip(y_pos, list(r2.iloc[4])):
    a5.text(x+0.001, y-0.03, int(y+0.5), ha='center', va='bottom')

fig.text(0.26, 0.95, 'Cluster Centroid LFRMC Characteristics')
fig.text(0.04, 0.5, 'Normalized Feature Value', va='center', rotation='vertical')
# fig.savefig('output/Cluster Centroid.png', dpi=500)
plt.show()

#### n_clusters = 3

In [ ]:
kmodel2 = KMeans(n_clusters=2, init='k-means++', 
                n_init=10,max_iter=500, tol=1e-04, 
                random_state=101)
kmodel_label2 = kmodel2.fit_predict(lrfmc_table)

In [ ]:
r4 = pd.Series(kmodel2.labels_).value_counts() 
r3 = pd.DataFrame(kmodel2.cluster_centers_) 
r3.columns = ['L', 'R', 'F', 'M', 'C']

In [ ]:
r3.iloc[0]

In [ ]:
y_pos = np.arange(len(r3.columns))

fig, axes = plt.subplots(1, 2, sharey=True)
a1 = sns.barplot(r3.columns, list(r3.iloc[0]), ax=axes[1], palette=('husl'))
a1.set_title(1)
for x, y in zip(y_pos, list(r3.iloc[0])):
    a1.text(x+0.001, y-0.03, '%.1f'%y, ha='center', va='bottom')

a2 = sns.barplot(r3.columns, list(r3.iloc[1]), ax=axes[0], palette='husl')
a2.set_title(2)
for x, y in zip(y_pos, list(r3.iloc[1])):
    a2.text(x+0.001, y-0.03, '%.1f'%y, ha='center', va='bottom')

fig.text(0.26, 0.95, 'Cluster Centroid LFRMC Characteristics')
fig.text(0.04, 0.5, 'Normalized Feature Value', va='center', rotation='vertical')
fig.savefig('output/Cluster Centroid2.png', dpi=500)
plt.show()

In [ ]:
# Assign the customer to a specific class
cluster_result = pd.concat([lrfmc_table, pd.Series(kmodel.labels_, 
                index=lrfmc_table.index)], axis=1) #详细输出每个样本对应的类别
cluster_result.columns = list(lrfmc_table.columns) + [u'Class']

In [ ]:
r = pd.concat([r2, r1], axis = 1)
r.columns = list(lrfmc_table) + [u'Size'] 
r = r.reset_index(drop=False)
r = r.rename(columns={'index':'Clus'})
r = r.set_index('Clus', drop=True)

In [ ]:
r['Perc'] = round(r.iloc[:,[5]].div(r.iloc[:,[5]].sum(axis=0), axis=1), 2)

In [ ]:
# r['Lifetime value'] = np.power(10, r['L']).astype(int)
# r['Last day'] = -np.power(10, (4-r['R'])).astype(int)
# r['Day count'] = np.power(10, r['F']).astype(int)
# r['Sum miles'] = np.power(10, r['M']).astype(int)
# r['Gross Profit'] = np.power(10, r['C']).astype(int)

In [ ]:
kmeans = KMeans(n_clusters=3, init='k-means++', 
                n_init=10,max_iter=500, tol=1e-04, 
                random_state=101).fit(lrfmc_table)
kmeans_pred = kmeans.predict(lrfmc_table)

### Radar map
- Visualize the clustering results by the radar map

In [ ]:
max = r2.values.max()
min = r2.values.min()

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, polar=True)
center_num = r.iloc[:, :6].values
feature = ["Lifetime", "Recency", "Frequency", "Total Miles", "Average Gross Profit"]
N = len(feature)

for i, v in enumerate(center_num):
    angles = np.linspace(0, 2 * np.pi, N, endpoint=False)
    center = np.concatenate((v[:-1], [v[0]]))
    angles = np.concatenate((angles, [angles[0]]))
    
    ax.plot(angles, center, 'o-', linewidth=2, label="Nb.%dClusters, %d Customers" % (i + 1, v[-1]))
    ax.fill(angles, center, alpha=0.25)
    ax.set_thetagrids(angles * 180 / np.pi, feature, fontsize=15)
    ax.set_ylim(min - 0.35, max + 0.35)
    plt.title('Cluster Segmentation Rader Map', fontsize=20)
    ax.grid(True)
    plt.legend(loc='upper right', bbox_to_anchor=(1.45, 1.0), ncol=1, fancybox=True, shadow=True)

# fig.savefig('RaderMap.png')
plt.show()

## Merge two dataset to a new dataset for the results explore

In [ ]:
CL_Bookings = pd.read_csv('Cj filtered.csv', header=0)

In [ ]:
index_cluster_result = cluster_result.reset_index(drop=False)

company_Class = index_cluster_result[['companyid', 'Class']]

In [ ]:
whole_dateset2 = pd.merge(CL_Bookings, company_Class, on='companyid', how='left')

In [ ]:
dataTransformed = dataTransformed.reset_index('companyid')
companyd_class = index_cluster_result.loc[:,['companyid', 'Class']]
dataTransformed = pd.merge(companyd_class, dataTransformed, on='companyid', how='left')

In [ ]:
cluster_0 = dataTransformed[dataTransformed['Class'] == 0]
cluster_1 = dataTransformed[dataTransformed['Class'] == 1]
cluster_2 = dataTransformed[dataTransformed['Class'] == 2]
cluster_3 = dataTransformed[dataTransformed['Class'] == 3]
cluster_4 = dataTransformed[dataTransformed['Class'] == 4]

In [ ]:
f, axes = plt.subplots(2, 2, figsize=(10, 10), dpi=80)
sns.distplot(cluster_0["L"] , color="royalblue", hist=False, ax=axes[0, 0], label='Class_0')
sns.distplot(cluster_1["L"] , color='green', hist=False,ax=axes[0, 0], label='Class=1')
sns.distplot(cluster_2["L"] , color='olive', hist=False,ax=axes[0, 0], label='Class=2')
sns.distplot(cluster_3["L"] , color='firebrick', hist=False,ax=axes[0, 0], label='Class=3')
sns.distplot(cluster_4["L"] , color='teal', hist=False,ax=axes[0, 0], label='Class=4')

sns.distplot(cluster_0["R"] , color="royalblue", hist=False, ax=axes[0, 1], label='Class_0')
sns.distplot(cluster_1["R"] , color='green', hist=False,ax=axes[0, 1], label='Class=1')
sns.distplot(cluster_2["R"] , color='olive', hist=False,ax=axes[0, 1], label='Class=2')
sns.distplot(cluster_3["R"] , color='firebrick', hist=False,ax=axes[0, 1], label='Class=3')
sns.distplot(cluster_4["R"] , color='teal', hist=False,ax=axes[0, 1], label='Class=4')

sns.distplot(cluster_0["F"] , color="royalblue", hist=False, ax=axes[1, 0], label='Class_0')
sns.distplot(cluster_1["F"] , color="green", hist=False, ax=axes[1, 0], label='Class_1')
sns.distplot(cluster_2["F"] , color="olive", hist=False, ax=axes[1, 0], label='Class_2')
sns.distplot(cluster_3["F"] , color="firebrick", hist=False, ax=axes[1, 0], label='Class_3')
sns.distplot(cluster_4["F"] , color="teal", hist=False, ax=axes[1, 0], label='Class_4')

sns.distplot(cluster_0["M"] , color="teal", ax=axes[1, 1])
sns.despine()

### Class=1(High net Industry)

In [ ]:
whole = whole_dateset2.groupby(['Industry'], as_index=False)['Class'].count()

In [ ]:
def class_nb (dataset, i):
    class_nb =whole_dateset2[(whole_dateset2['Class'] == i)][['Class', 'Industry']]
    class_nb = class_nb.groupby(['Industry'], as_index=False)['Class'].count()
    class_nb = class_nb.rename(columns = {'Class':'Count'})
    class_nb = pd.merge(class_nb, whole, on='Industry', how='left')
    class_nb['Perc'] = np.round(class_nb['Count'] / class_nb['Class'], 2)
    class_nb = class_nb.sort_values(by='Perc', ascending=False)
    return class_nb

In [ ]:
class_1 = class_nb(whole_dateset2, 0)

In [ ]:
plt.figure(figsize=(9, 9), dpi=80)
g = sns.barplot(x="Perc", y="Industry", data=class_1, palette='Blues_d')
g.set_title("Class = 1 (High Net) Industry Distribution", fontweight='bold')
sns.despine()

### Class=2 (General)

In [ ]:
class_2 = class_nb(whole_dateset2, 1)

In [ ]:
plt.figure(figsize=(9, 10), dpi=80)
g = sns.barplot(x="Perc", y="Industry", data=class_2, palette='Blues_d')
g.set_title("Class = 2 (General) Industry Distribution", fontweight='bold')
sns.despine()

### Class=3 (Retention) 

In [ ]:
class_3 = class_nb(whole_dateset2, 2)

In [ ]:
plt.figure(figsize=(9, 10), dpi=80)
g = sns.barplot(x="Perc", y="Industry", data=class_3, palette='Blues_d')
g.set_title("Class = 3 (Retention) Industry Distribution", fontweight='bold')
sns.despine()

### Class=4 (High Potential) 

In [ ]:
class_4 = class_nb(whole_dateset2, 3)

In [ ]:
plt.figure(figsize=(9,10), dpi=80)
g = sns.barplot(x="Perc", y="Industry", data=class_4, palette='Blues_d')
g.set_title("Class = 4 (High Potential) Industry Distribution", fontweight='bold')
sns.despine()

### Class=5 (Low Net) 

In [ ]:
class_5 = class_nb(whole_dateset2, 4)

In [ ]:
plt.figure(figsize=(9,9), dpi=80)
g = sns.barplot(x="Perc", y="Industry", data=class_5, palette='Blues_d')
g.set_title("Class = 5 (Low Net) Industry Distribution", fontweight='bold')
sns.despine()

### Explore results from the companyid

In [ ]:
def company_num(dataset, i):
    company_nb = dataset[(dataset['Class'] == i)][['Class', 'companyid']]
    company_nb = company_nb.groupby(['companyid'], as_index=False)['Class'].count()
    company_nb = company_nb.rename(columns = {'Class':'Count'})
    company_nb['Perc'] = company_nb['Count'] / np.sum(company_nb['Count'], axis=0)
    company_nb_head = company_nb.sort_values(by='Count', ascending=False)[:10]
    
    y_pos = np.array(company_nb_head['companyid'])
    performance = np.array(company_nb_head['Count'])
    
    cols = []
    for i in range(len(company_nb_head)):
        col = 'company_{}'.format(y_pos[i])
        cols.append(col)
        
    return company_nb_head, y_pos, performance, cols

In [ ]:
company_1_class, y_pos, performance, cols = company_num(whole_dateset2, 0)

In [ ]:
plt.figure(figsize=(10, 8), dpi=80)
g = sns.barplot(performance,cols, palette='GnBu_d')
g.set_title("Class = 1 (High New) Companyd Distribution")
sns.despine()

In [ ]:
company_2_c, y_pos, performance, cols = company_num(whole_dateset2, 1)

In [ ]:
plt.figure(figsize=(10, 8), dpi=80)
g = sns.barplot(performance, cols, palette='GnBu_d')
g.set_title("Class = 2 (Retention) Companyd Distribution")
sns.despine()

In [ ]:
company_3_class, y_pos, performance, cols = company_num(whole_dateset2, 2)

In [ ]:
plt.figure(figsize=(10, 8), dpi=80)
g = sns.barplot(performance, cols, palette='GnBu_d')
g.set_title("Class = 3 (Retention) Companyd Distribution")
sns.despine()

In [ ]:
company_4_class, y_pos, performance, cols = company_num(whole_dateset2, 3)

In [ ]:
plt.figure(figsize=(10, 8), dpi=80)
g = sns.barplot(performance, cols, palette='GnBu_d')
g.set_title("Class = 4 (High Potential) Companyd Distribution")
sns.despine()

In [ ]:
company_5_class, y_pos, performance, cols = company_num(whole_dateset2, 4)

In [ ]:
plt.figure(figsize=(10, 8), dpi=80)
g = sns.barplot(performance, cols, palette='GnBu_d')
g.set_title("Class = 5 (Low net) Companyd Distribution")
sns.despine()

### Merge two dataset to a new dataset for the final modeling

In [ ]:
CL_Model = pd.read_csv('CL dataset.csv', header=0)

In [ ]:
whole_dateset1 = pd.merge(CL_Model, company_Class, on='companyid', how='left')

In [ ]:
whole_dateset1.to_csv('whole_dataset.csv', index=False)

### Clustermap

In [ ]:
import matplotlib.gridspec
sns.set(color_codes=True)

In [ ]:
Class = cluster_result.pop("Class")

In [ ]:
lut = dict(zip(Class.unique(), "rbg"))
row_colors = Class.map(lut)

#First create the clustermap figure
g = sns.clustermap(cluster_result, row_colors = row_colors, figsize=(13,8))
# set the gridspec to only cover half of the figure
g.gs.update(left=0.05, right=0.45)

#create new gridspec for the right part
gs2 = matplotlib.gridspec.GridSpec(1,1, left=0.6)
# create axes within this new gridspec
ax2 = g.fig.add_subplot(gs2[0])
# plot boxplot in the new axes
sns.boxplot(data=cluster_result, orient="h", palette="Set2", ax = ax2)
fig.savefig('clustermap.png')
plt.show()

# ANOVA testing for feature selection

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [ ]:
whole_dateset1.shape

In [ ]:
whole_dateset1 = whole_dateset1.drop(['companyid'], axis=1)

cols = [i for i in whole_dateset1.columns if i not in ['ProfitMargin(%)']]

In [ ]:
model = sm.OLS(endog = whole_dateset1['ProfitMargin(%)'].astype(int), exog = whole_dateset1[cols]).fit()
print(model.summary())

In [ ]:
whole_dateset1 = whole_dateset1.rename(columns = {'Product_Tour Ops': 'Product_Tour', 'ProfitMargin(%)':'ProfitMargin', 'Country_United Kingdom':'Country_UK', 'Country_United States':'Country_US'})

In [ ]:
formula = 'ProfitMargin ~ HashEncode_0 + HashEncode_1 + HashEncode_2 + HashEncode_3 + HashEncode_4 + HashEncode_5 + HashEncode_6 + HashEncode_7 + \
           Round_trip + Scaled_gross_log + Scaled_Distance_log + Scaled_FlightTime_log + Country_France + Country_Germany + Country_Italy + Country_Monaco + Country_Other + Country_Sweden + \
           Country_Switzerland + Country_Turkey + Country_UK + Country_US + Product_Tour + passengersRange_1 + passengersRange_2 + \
           passengersRange_3 + categ_aircraft + categ_0 + categ_1 + categ_2 + categ_3 + categ_4 + categ_5 + Class'
results = ols(formula, data = whole_dateset1).fit()

In [ ]:
results.summary()

In [ ]:
aov_table = sm.stats.anova_lm(results, typ=2)
aov_table